# Generative AI-powered search with Amazon OpenSearch Service

---


In this lab, we leverage LangChain framework to implement agent based chatbox application.

In RAG, external data can be sourced from various data sources, such as document repositories, databases, or APIs. The first step is to convert the documents and the user query into a format that enables comparison and allows for performing relevancy search. To achieve comparability for relevancy search, a document collection (knowledge library) and the user-submitted query are transformed into numerical representations using embedding language models. These embeddings are essentially numerical representations of concepts in text.

Next, based on the embedding of the user query, relevant text is identified in the document collection through similarity search in the embedding space. The prompt provided by the user is then combined with the searched relevant text and added to the context. This updated prompt, which includes relevant external data along with the original prompt, is sent to the LLM (Language Model) for processing. As a result, the model output becomes relevant and accurate due to the context containing the relevant external data.

For more informaiton about LangChain RAG, please refer: https://python.langchain.com/docs/use_cases/question_answering/

---

The lab includes the following steps:
- [Initialize](#Initialize)
- [Part 1: Ingest unstructured data into OpenSearch](#Part-1:-Ingest-unstructured-data-into-OpenSearch)
- [Part 2: Different appoach to search](#Part-2:-Different-appoach-to-search)
    - [2.1 Keyword search](#2.1-Keyword-search)
    - [2.2 Semantic/Vector search](#2.2-Semantic/Vector-search)
    - [2.3 Hybrid search](#2.3-Hybrid-search)
    - [2.4 Retrieval Augmented Generation(RAG)](#2.4-Retrieval-Augmented-Generation(RAG))
    - [2.5 Conversational search](#2.5-Conversational-search)
- [Part 3: Agent powered search](#Part-3:-Agent-powered-search)
    - [3.1 Ingest structured data into Redshift](#3.1-Ingest-structured-data-into-Redshift)
    - [3.2 Query unstructured data in OpenSearch with vector search](#3.2-Query-unstructured-data-in-OpenSearch-with-vector-search)
    - [3.3 Query structured data in Redshift with SQLDatabaseChain](#3.3-Query-structured-data-in-Redshift-with-SQLDatabaseChain)
    - [3.4 Create LLM Based ReAct Agent Augmented with Data in OpenSearch and Redshift](#3.4-Create-LLM-Based-ReAct-Agent-Augmented-with-Data-in-OpenSearch-and-Redshift)
    - [3.5 Use LLM based agent](#3.5-Use-LLM-based-agent)



### Check PyTorch Version
Make sure PyTorch versin is larger than or equal  2.2.0

In [ ]:
import torch
print(torch.__version__)

## Initialize




###  Install OpenSearch ML Python library

In [ ]:
!pip install -q opensearch-py
%pip install langchain
%pip install boto3
%pip install sqlalchemy>
%pip install sqlalchemy-redshift
%pip install redshift_connector
%pip install ipython-sql==0.4.1
%pip install langchain_experimental

### Import library



In [ ]:
import boto3
import re
import time
import sagemaker,json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

## Part 1: Ingest unstructured data into OpenSearch

### Datasets
Form 10-K is a comprehensive report filed annually by a publicly traded company about its financial performance and is required by the U.S. Securities and Exchange Commission (SEC). Some of the information a company is required to document in the 10-K includes its history, organizational structure, financial statements, earnings per share, subsidiaries, executive compensation, and any other relevant data.

The SEC mandates that all public companies file regular 10-Ks to keep investors aware of a company's financial condition and to allow them to have enough information before they buy or sell securities issued by that company. The 10-K can appear overly complex at first glance, complete with tables full of data and figures. However, it is so comprehensive that this filing is critical for investors to handle a company's financial position and prospects.

Form 10-K is an annual report that provides a comprehensive analysis of the company's financial condition. The Form 10-K is comprised of several parts. These include:

#### Business summary:
This describes the company's operations. It would include information about business segments, products and services, subsidiaries, markets, regulatory issues, research and development, competition, and employees, among other details.

#### Management Discussion and Analysis:
This section allows the company to explain its operations and financial results for the past year.

#### Financial statements:
The financial statements would include the company's balance sheet, income statement, and cash flow statement.

Additional sections: Additional sections may discuss the company's management team and legal proceedings.

### 10K Form

- 1 - Business
- 1A - Risk Factors
- 1B - Unresolved Staff Comments
- 1C - Cybersecurity
- 2 - Properties
- 3 - Legal Proceedings
- 4 - Mine Safety Disclosures
- 5 - Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
- 6 - Selected Financial Data (prior to February 2021)
- 7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
- 7A - Quantitative and Qualitative Disclosures about Market Risk
- 8 - Financial Statements and Supplementary Data
- 9 - Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
- 9A - Controls and Procedures
- 9B - Other Information
- 10 - Directors, Executive Officers and Corporate Governance
- 11 - Executive Compensation
- 12 - Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
- 13 - Certain Relationships and Related Transactions, and Director Independence
- 14 - Principal Accountant Fees and Services
- 15 - Exhibits and Financial Statement Schedules

In [ ]:
!pip install sec-api

### SEC API
https://sec-api.io/docs/query-api

In [ ]:
sec_api_key="510e614fa84ef39a9de2012838bfbef33d4e1f1d60c26d4765441fad945d9adf"

In [ ]:
from sec_api import QueryApi
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

queryApi = QueryApi(api_key=sec_api_key)
query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" AND ticker:AMZN", # only 10-Ks
  }},
  "from": "0", # start returning matches from position null, i.e. the first matching filing 
  "size": "3"  # return just one filing
}

response = queryApi.get_filings(query)
metadata = pd.DataFrame.from_records(response['filings'])
metadata.head(3)

In [ ]:
from sec_api import RenderApi
import os
import multiprocessing
import time

# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "w")


# for each filing, only save the URL pointing to the filing itself 
# and ignore all other data. 
# the URL is set in the dict key "linkToFilingDetails"
urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))

# transform list of URLs into one string by joining all list elements
# and add a new-line character between each element.
urls_string = "\n".join(urls_list)

log_file.write(urls_string)

log_file.close()


renderApi = RenderApi(api_key=sec_api_key)

def download_filing(url):
  try:
    filing = renderApi.get_filing(url)
    file_name = url.split("/")[-2] + "-" + url.split("/")[-1] 
    download_to = "./filings/" + file_name
    with open(download_to, "w") as f:
      f.write(filing)
    time.sleep(2)
  except Exception as e:
    print("Problem with {url}".format(url=url))
    print(e)
    
# load URLs from log file
def load_urls():
  log_file = open("filing_urls.txt", "r")
  urls = log_file.read().split("\n") # convert long string of URLs into a list 
  log_file.close()
  return urls


def download_all_filings():
  print("Start downloading all filings")

  download_folder = "./filings" 
  if not os.path.isdir(download_folder):
    os.makedirs(download_folder)
    
  urls = load_urls()
  print("{length} filing URLs loaded".format(length=len(urls)))

  number_of_processes = 20

  with multiprocessing.Pool(number_of_processes) as pool:
    pool.map(download_filing, urls)
  


In [ ]:
download_all_filings()

### Get SEC 10K from downloaded files

In [ ]:
!wget https://ws-assets-prod-iad-r-sfo-f61fc67057535f1b.s3.us-west-1.amazonaws.com/df655552-1e61-4a6b-9dc4-c03eb94c6f75/10k-financial-filing.zip

Unzip the dataset

In [ ]:
#!unzip 10k-financial-filing.zip

Read the dataset in JSON format and contruct pandas DataFrame

In [ ]:
# Specify the path to the folder containing the JSON files
folder_path = "extracted"

# Initialize an empty list to store DataFrames
dataframes = []

#For this session, we only ingest few company information.
company_list=["Alteryx, Inc.", "MICROSTRATEGY Inc", 
              "Elastic N.V.", "MongoDB, Inc.", 
              "Palo Alto Networks Inc", "Okta, Inc.",
              "Datadog, Inc.", "Snowflake Inc.",
              "SALESFORCE.COM, INC.", "ORACLE CORP",
              "MICROSOFT CORP", "Palantir Technologies Inc."
             ]

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        df = pd.DataFrame([pd.read_json(file_path,typ='series')])
        if df.iloc[0]['company'] in company_list:
            dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)



To display whole DataFrame

In [ ]:
combined_df

### Create an OpenSearch cluster connection.
Next, we'll use Python API to set up connection with OpenSearch Cluster.

Note: if you're using a region other than us-east-1, please update the region in the code below.

#### Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
import json
region = aws_region

cfn = boto3.client('cloudformation')
kms = boto3.client('secretsmanager')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "generative-ai-powered-search"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['OpenSearchSecret'])['SecretString'])

outputs

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection

auth = (aos_credentials['username'], aos_credentials['password'])
aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

### Create a index in Amazon Opensearch Service 
Whereas we previously created an index with 2 fields, this time we'll define the index with 3 fields: the first field ' question_vector' holds the vector representation of the question, the second is the "question" for raw sentence and the third field is "answer" for the raw answer data.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil"
    },
    "mappings": {
        "properties": {
            "item_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "store": True,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {
                      "ef_construction": 128,
                      "m": 24
                    }
                }
            },
            "item_content": {
                "type": "text",
                "store": True
            },
            "company_name": {
                "type": "text",
                "store": True
            }
        }
    }
}

index_name="10k_financial"

Using the above index definition, we now need to create the index in Amazon OpenSearch

In [ ]:
aos_client.indices.create(index=index_name,body=knn_index,ignore=400)

Let's verify the created index information

In [ ]:
aos_client.indices.get(index=index_name)

In [ ]:
from langchain.embeddings import BedrockEmbeddings


boto3_bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=f"https://bedrock-runtime.{aws_region}.amazonaws.com")
bedrock_embeddings = BedrockEmbeddings(model_id='amazon.titan-embed-text-v1',client=boto3_bedrock)
result = bedrock_embeddings.embed_query("This is a content of the document")


In [ ]:
result

###  Load the raw data into the Index
Next, let's load the financial billing data into the index we've just created.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import pandas

from typing import Any, Dict, List, Optional, Sequence

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

class PandasDataFrameLoader(BaseLoader):
    
    def __init__(self,dataframe:pandas.DataFrame):
        self.dataframe=dataframe
        
    def load(self) -> List[Document]:
        docs = []
        items=["item_1","item_1A","item_1B","item_2","item_3","item_4","item_5","item_6","item_7","item_7A","item_8","item_9","item_9A", "item_9B", "item_10", "item_11", "item_12", "item_13", "item_14", "item_15"]
        
        for index, row in self.dataframe.iterrows():
            metadata={}
            metadata["cik"]=row['cik']
            metadata["company_name"]=row['company']
            metadata["filing_date"]=row['filing_date']
            for item in items:
                content=row[item]
                metadata['item'] = item
                doc = Document(page_content=content,metadata=metadata)
                #print(doc.metadata)
                docs.append(doc)
        return docs
    
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 8000, chunk_overlap = 200)
pd_loader = PandasDataFrameLoader(combined_df)
documents = pd_loader.load()
#splitted_documnets = pd_loader.load_and_split(text_splitter=text_splitter)
splitted_documnets = text_splitter.split_documents(documents)


Use Bedrock embedding convert item content into vector and use OpenSearch bulk ingest to store data into OpenSearch index

In [ ]:
import time
from opensearchpy import helpers

previous_company_name=""
item_contents=[]
for doc in splitted_documnets:
    company_name=doc.metadata['company_name']
    if previous_company_name != "" and previous_company_name != company_name:
        print("company:" + company_name + ", item count:" + str(len(item_contents)))
        start = time.time()
        embedding_results = bedrock_embeddings.embed_documents(item_contents)
        end = time.time()
        elapsed = end - start
        print(f"total time elapsed for Bedrock embedding: {elapsed:.2f} seconds")
        #TODO
        data = []
        i=0
        for content in item_contents:
            data.append({"_index": index_name,  "company_name": previous_company_name, "item_content":content, "item_vector":embedding_results[i]})
            i = i+1
        aos_response= helpers.bulk(aos_client, data)
        print(f"Bulk-inserted {aos_response[0]} items.")
        item_contents=[]
    
    item_contents.append(doc.page_content)
    previous_company_name=company_name


To validate the load, we'll query the number of documents number in the index. 

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])

In [ ]:
aos_client.indices.delete(index=index_name)


## Part 2: Different appoach to search

### 2.1 Keyword search

### 2.2 Semantic/Vector search

### 2.3 Hybrid search

### 2.4 Retrieval Augmented Generation(RAG)

### 2.5 Conversational search

## Part 3: Agent powered search

### 3.1 Ingest structured data into Redshift

Get Redshift Serverless username, password and endpoint

In [ ]:
redshift_serverless_credentials = json.loads(kms.get_secret_value(SecretId=outputs['RedshiftServerlessSecret'])['SecretString'])
redshift_serverless_username=redshift_serverless_credentials['username']
redshift_serverless_password=redshift_serverless_credentials['password']
redshift_serverless_endpoint =  outputs['RedshiftServerlessEndpoint']

connect to Redshift

In [ ]:
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import Session
%reload_ext sql
%config SqlMagic.displaylimit = 25

connect_to_db = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=redshift_serverless_endpoint, 
port=5439,
database='dev',
username=redshift_serverless_username,
password=redshift_serverless_password
)

%sql $connect_to_db
%sql select current_user, version();


### Populate the Redshift table 

### Stock symbol table

create `stock_symbol` table

In [ ]:
%sql CREATE TABLE IF NOT EXISTS public.stock_symbol (stock_symbol text PRIMARY KEY, company_name text NOT NULL);


insert data

In [ ]:
stock_price_bucket = outputs["s3BucketStock"]
s3_location = f's3://{stock_price_bucket}/stock-price/'
print(s3_location)
!aws s3 sync ./stock-price/ $s3_location

stock_symbol_s3_location = f's3://{stock_price_bucket}/stock-price/stock_symbol.csv'
print(stock_symbol_s3_location)
quoted_stock_symbol_s3_location = "'" + stock_symbol_s3_location + "'"
print(quoted_stock_symbol_s3_location)
print("---------")



In [ ]:
%sql COPY STOCK_SYMBOL FROM $quoted_stock_symbol_s3_location iam_role default IGNOREHEADER 1 CSV;


query `stock_symbol` table

In [ ]:
%sql select * from public.stock_symbol

### Stock price table

Create stock price table

In [ ]:
%sql CREATE TABLE IF NOT EXISTS public.stock_price (stock_date DATE, stock_symbol text, open_price DECIMAL, high_price DECIMAL, low_price DECIMAL, close_price DECIMAL, adjusted_close_price DECIMAL, volume DECIMAL);



In [ ]:
msft_s3_location = f's3://{stock_price_bucket}/stock-price/MSFT.csv'
print(msft_s3_location)
quoted_msft_s3_location = "'" + msft_s3_location + "'"
print(quoted_msft_s3_location)
print("---------")

crm_s3_location = f's3://{stock_price_bucket}/stock-price/CRM.csv'
print(crm_s3_location)
quoted_crm_s3_location = "'" + crm_s3_location + "'"
print(quoted_crm_s3_location)
print("---------")

orcl_s3_location = f's3://{stock_price_bucket}/stock-price/ORCL.csv'
print(orcl_s3_location)
quoted_orcl_s3_location = "'" + orcl_s3_location + "'"
print(quoted_orcl_s3_location)
print("---------")

snow_s3_location = f's3://{stock_price_bucket}/stock-price/SNOW.csv'
print(snow_s3_location)
quoted_snow_s3_location = "'" + snow_s3_location + "'"
print(quoted_snow_s3_location)
print("---------")


In [ ]:
%sql COPY STOCK_PRICE FROM $quoted_msft_s3_location iam_role default IGNOREHEADER 1 CSV;
%sql COPY STOCK_PRICE FROM $quoted_crm_s3_location iam_role default IGNOREHEADER 1 CSV;
%sql COPY STOCK_PRICE FROM $quoted_orcl_s3_location iam_role default IGNOREHEADER 1 CSV;
%sql COPY STOCK_PRICE FROM $quoted_snow_s3_location iam_role default IGNOREHEADER 1 CSV;

In [ ]:
%sql select * from public.stock_price

### 3.2 Query unstructured data in OpenSearch with vector search

Define semantic search function

In [ ]:
def semantic_search(query, k=10):
    print("semantic search input: " + query)
    search_vector = bedrock_embeddings.embed_query(query)
    query={
        "size": 10,
        "query": {
            "knn": {
                "item_vector":{
                    "vector":search_vector,
                    "k":10
                }
            }
        }
    }

    res = aos_client.search(index=index_name, 
                       body=query,
                       stored_fields=["company_name","item_category","item_content"])
    #print("Got %d Hits:" % res['hits']['total']['value'])
    query_result=[]
    for hit in res['hits']['hits']:
        row=[hit['fields']['item_content'][0]]
        print(row)
        query_result.append(row)

    query_result_df = pd.DataFrame(data=query_result,columns=["item_content"])
    return query_result_df

def semantic_search_full_field(query, k=5):
    search_vector = bedrock_embeddings.embed_query(query)
    query={
        "size": 5,
        "query": {
            "knn": {
                "item_vector":{
                    "vector":search_vector,
                    "k":10
                }
            }
        }
    }

    res = aos_client.search(index=index_name, 
                       body=query,
                       stored_fields=["company_name","item_category","item_content"])
    #print("Got %d Hits:" % res['hits']['total']['value'])
    query_result=[]
    for hit in res['hits']['hits']:
        row=[hit['_id'],hit['_score'],hit['fields']['company_name'][0],hit['fields']['item_category'][0],hit['fields']['item_content'][0]]
        query_result.append(row)

    query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","company_name","item_category","item_content"])
    return query_result_df


In [ ]:
def get_financial_statements(query):
    company_statements = semantic_search(query)
    return company_statements

In [ ]:
semantic_search_result=get_financial_statements("Is Microsoft a good investment choice right now?")

In [ ]:
pd.set_option('display.max_colwidth', None)
semantic_search_result.head(5)

### 3.3 Query structured data in Redshift with SQLDatabaseChain
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

#### SQL Database Chain
Langchain has a SQL Database chain to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/use_cases/qa_structured/sql#case-2-text-to-sql-query-and-execution


Initialize Bedrock LLM model with Claude

In [ ]:
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun

#bedrock_llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock)
bedrock_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock)

bedrock_llm.model_kwargs = {"max_tokens_to_sample":1204,"temperature":0.01,"top_k":250,"top_p":1,"stop_sequences":["\\n\\nHuman:"]}


#### Get stock symbol

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=redshift_serverless_endpoint, 
port=5439,
database='dev',
username=redshift_serverless_username,
password=redshift_serverless_password
)


db = SQLDatabase.from_uri(url,include_tables=['stock_symbol'])

from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Human: Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
If the results of the query is empty, answer \"I don't know\"
<format>
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Answer with SQLResult. If SQLResult is empty, asnwer I don't know"
</format>
Assistant: Understood, I will use the above format and only provide the answer.

Only use the following tables:
<tables>
CREATE TABLE stock_symbol (
	stock_symbol text PRIMARY KEY,
	company_name text NOT NULL
)
</tables>

If someone asks for the table stock symbol table, they really mean the stock_symbol table.
<examples>
Question: What is the ticker symbol for Amazon in stock symbol table?
SQLQuery: SELECT stock_symbol FROM stock_symbol WHERE lower(company_name) ILIKE '%Amazon%'
SQLResult: AMZN
Answer: AMZN

Question: What is the ticker symbol for Microsoft in stock ticker table?
SQLQuery: SELECT stock_symbol FROM stock_symbol WHERE lower(company_name) ILIKE '%Microsoft%'
SQLResult: empty
Answer: I don't know

</examples>

Question: {input}

"""

PROMPT = PromptTemplate(
    input_variables=["input", "dialect"], template=_DEFAULT_TEMPLATE
)

llm=bedrock_llm

db_chain = SQLDatabaseChain.from_llm(
    llm, 
    db, 
    verbose=True, 
    return_intermediate_steps=True, 
    prompt=PROMPT, 
    )


Get "Amazon" stock symbol

In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for Amazon in stock symbol table? \n\nAssistant:")
response['result']

Get "Microsoft" stock symbol

In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for MICROSOFT in stock ticker table? \n\nAssistant:")
response['result']

Try to get a non exist company stock symbol, it will return "I don't know"

In [ ]:
response = db_chain("\n\nHuman: What is the ticker symbol for Pan Test in stock ticker table? \n\nAssistant:")
response['result']

#### Get stock price

In [ ]:
stock_price_template = """Human: Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
If the results of the query is empty, answer \"I don't know\"
<format>
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Answer with SQLResult. If SQLResult is empty, asnwer I don't know"
</format>
Assistant: Understood, I will use the above format and only provide the answer.

Only use the following tables:
<tables>
CREATE TABLE public.stock_price (
stock_date DATE, 
stock_symbol text, 
open_price DECIMAL, 
high_price DECIMAL, 
low_price DECIMAL, 
close_price DECIMAL, 
adjusted_close_price DECIMAL, 
volume DECIMAL);
</tables>

answer the following question and organize the return data into json format.
1. what is the company average open price in the month of July? 
2. what is the company average close price in the month of July?
3. what is the company average high price in the month of July?
4. what is the company average low price in the month of July?
5. what is the company average adjusted close price in the month of July?
6. what is the company average volume in the month of July?

Question: {input}

"""

stock_prompt = PromptTemplate(
    input_variables=["input", "dialect"], template=stock_price_template
)

llm=bedrock_llm

stock_price_db_chain = SQLDatabaseChain.from_llm(
    llm, 
    db, 
    verbose=True, 
    return_intermediate_steps=True, 
    prompt=stock_prompt, 
    )


Get MSFT stock price

In [ ]:
stock_price_info = stock_price_db_chain("\n\nHuman: What is the MSFT stock information? \n\nAssistant:")

In [ ]:
print(stock_price_info['result'])

Define functions:
1. if query is stock related, 
2. get stock ticker from the query
3. get stock price

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

def is_stock_related_query(query):
    template = """You are a helpful assistant to judge if the human input is stock related question.
    If it is stock related, answer \"yes\". Otherwise answer \"no\"."""
    human_template = "{text}"

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", human_template),
    ])

    llm_chain = LLMChain(
        llm=llm,
        prompt=chat_prompt
    )
    stock_related = llm_chain({"text":query})['text'].strip()
    return stock_related, query
    
def get_stock_ticker(query):
    template = """You are a helpful assistant who extract company name from the human input.Please only output the company"""
    human_template = "{text}"

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", human_template),
    ])

    llm_chain = LLMChain(
        llm=llm,
        prompt=chat_prompt
    )

    company_name=llm_chain(query)['text'].strip()
    company_ticker = db_chain("\n\nHuman: What is the ticker symbol for " + str(company_name) + " in stock ticker table? \n\nAssistant:")
    return company_name, company_ticker['result']

def get_stock_price(ticker):
    #get stock price with text to sql in db_chain
    data = stock_price_db_chain("\n\nHuman: What is the " + str(ticker) + " stock information ? \n\nAssistant:")
    return data['result']


### 3.4 Create LLM Based ReAct Agent Augmented with Data in OpenSearch and Redshift

Define agent tools

In [ ]:
from langchain.agents import Tool, AgentType

tools=[
    Tool(
        name="is stock related query",
        func=is_stock_related_query,
        description="If the query is stock related"
    ),
    Tool(
        name="get company ticker",
        func=get_stock_ticker,
        description="Get the company stock ticker"
    ),
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data company's historic performance can be evaluated. You should input stock ticker to it"
    ) 
]


define agent prompt

In [ ]:
updated_prompt="""Human: You are a financial advisor. Give stock recommendations for given query based on following instructions. 
<instructions>
Answer the following questions as best you can. You have access to the following tools:

is stock related query: Use when you need to know whether this is stock related query. This tool will output whether human input is stock related and human input. You should input the human input to it.
get company ticker: Use when you need to extract company name and stock ticker. This tool will output company name and stock ticker.
get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the stock ticker to it.
get financial statements: Use this to get financial statement of the company. With the help of this data, companys historic performance can be evaluated. You should input the human input to it.
</instructions>

<steps>
Note- if you fail in satisfying any of the step below, Just move to next one
1) Use "is stock related query" tool to judge if the input query is stock related or not. Output - stock related and input query
2) Use "get company ticker" tool to get the company name and stock ticker. Output- company name and stock ticker
3) Use "get stock data" tool to gather stock info. Output- Stock data
4) Use "get financial statements" tool to get company's historic financial statement. Output- Financial statement
5) Analyze the stock based on gathered data and give detail analysis for investment choice. provide numbers and reasons to justify your answer. 
If there is no output from "get stock data" tool, please ouput "I cannot provide stock analysis without stock price information.".
Output- Detailed stock Analysis
</steps>

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [is stock related query, get company ticker, get stock data, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}

"""

Initialize agent

In [ ]:
from langchain.agents import initialize_agent 

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=2,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

zero_shot_agent.agent.llm_chain.prompt.template=updated_prompt

### 3.5 Use LLM based agent

#### Example 1:

Ask the queustion "Is Microsoft a good investment choice right now?". The agent will run the following process:

1. is stock related query
2. get company name
3. get stock symbol
4. get stock price
5. use semantic search get related information from 10k financial filing data

Combine all the above information and generate answer.

In [ ]:
import warnings
warnings.filterwarnings("ignore")


response = zero_shot_agent("\n\nHuman: Is Microsoft a good investment choice right now? \n\nAssistant:")

In [ ]:
print(response["output"])

In [ ]:
print(response["intermediate_steps"])

#### Example 2

"Is Amazon a good investment choice right now?". This is stock related quesiton. However there is no Amazon stock price information in Redshift table. So it will answer "I cannot provide stock analysis without stock price information." in the end. The agent will run the following process:

1. is stock related query
2. get company name
3. get stock symbol
4. get stock price

In [ ]:
response = zero_shot_agent("\n\nHuman: Is Amazon a good investment choice right now? \n\nAssistant:")


In [ ]:
print(response["output"])

#### Example 3

This is not stock related query. The agent will run the following process:

1. is stock related query

In [ ]:
response = zero_shot_agent("\n\nHuman: What is SageMaker? \n\nAssistant:")

In [ ]:
print(response["output"])